## Создаем подключение к базе данных с помощью библиотеки sqlite3

In [80]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('../data/checking-logs.sqlite')
cursor = conn.cursor()

## Используя только один запрос для каждой из групп, создайте два фрейма данных: test_results и control_results со столбцами time и avg_diff и только двумя строками
- time должно иметь значения: после и до
- avg_diff содержит среднюю дельту среди всех пользователей за период времени до того, как каждый из них совершил свой первый визит на страницу, и после этого
- учитывать только тех пользователей, у которых есть наблюдения до и после
- мы все еще не используем лабораторию «project1»

In [81]:
test_query = """
SELECT
    'after' AS time,
    -AVG((julianday(datetime(deadlines.deadlines, 'unixepoch')) - julianday(test.first_commit_ts)) * 24) AS avg_diff
FROM
    test
JOIN
    deadlines ON test.labname = deadlines.labs
WHERE
    deadlines.labs != 'project1'
    AND test.first_commit_ts IS NOT NULL
    AND deadlines.deadlines IS NOT NULL
    AND test.first_commit_ts >= test.first_view_ts

UNION ALL

SELECT
    'before' AS time,
    -AVG((julianday(datetime(deadlines.deadlines, 'unixepoch')) - julianday(test.first_commit_ts)) * 24) AS avg_diff
FROM
    test
JOIN
    deadlines ON test.labname = deadlines.labs
WHERE
    deadlines.labs != 'project1'
    AND test.first_commit_ts IS NOT NULL
    AND deadlines.deadlines IS NOT NULL
    AND test.first_commit_ts < test.first_view_ts;
"""


test_results = pd.read_sql_query(test_query, conn)

print("Test Results:")
print(test_results)

Test Results:
     time    avg_diff
0   after -103.953310
1  before  -61.156438


In [82]:
control_query = """
SELECT
    'after' AS time,
    -AVG((julianday(datetime(deadlines.deadlines, 'unixepoch')) - julianday(control.first_commit_ts)) * 24) AS avg_diff
FROM
    control
JOIN
    deadlines ON control.labname = deadlines.labs
WHERE
    deadlines.labs != 'project1'
    AND control.first_commit_ts IS NOT NULL
    AND deadlines.deadlines IS NOT NULL
    AND control.first_commit_ts >= control.first_view_ts

UNION ALL

SELECT
    'before' AS time,
    -AVG((julianday(datetime(deadlines.deadlines, 'unixepoch')) - julianday(control.first_commit_ts)) * 24) AS avg_diff
FROM
    control
JOIN
    deadlines ON control.labname = deadlines.labs
WHERE
    deadlines.labs != 'project1'
    AND control.first_commit_ts IS NOT NULL
    AND deadlines.deadlines IS NOT NULL
    AND control.first_commit_ts < control.first_view_ts;
"""

control_results = pd.read_sql_query(control_query, conn)

print("\nControl Results:")
print(control_results)


Control Results:
     time    avg_diff
0   after -113.232197
1  before  -99.901295


## Закрыть соединение

In [83]:
conn.close()